# intro to pandas

This file is a [Jupyter](https://jupyter.org/) notebook. The output that appears here was created by a Python kernel when this page was created. You can type the commands that appear in a notebook file like this one into your Python shell (or run them in a Python script) and expect to see the same results, assuming you have the dependencies installed.

We'll be taking a look at a **library** called [pandas](http://pandas.pydata.org/) which gives us some important basic functionality for handling datasets in Python.

If you're not sure what commands are available to you, note that (like the Unix command line) iPython supports **tab-completion**.

## 3. editing data

Let's import pandas and load our DataFrame again.

In [79]:
import pandas as pd
HTS_FILE = '~/gits/gads_26/datasets/state_hts.tsv'
data = pd.read_csv(HTS_FILE, sep='\t')

We can **create a new column** by simply assigning values to it. Recall that write operations to columns require element syntax, not attribute syntax.

In [80]:
FEET_TO_METERS = 0.3048
data['elev_m'] = FEET_TO_METERS * data.elev_ft
data.head()

,state,peak,elev_ft,elev_m
0,Alabama,Cheaha Mountain,2405,733.0440
1,Alaska,Denali,20320,6193.5360
2,Arizona,Humphreys Peak,12633,3850.5384
3,Arkansas,Magazine Mountain,2753,839.1144
4,California,Mount Whitney,14495,4418.0760


Another method for **creating one column from another** uses the Series `apply` method. This gives us greater flexibility:

In [81]:
data['elev_m'] = data.elev_ft.apply(lambda k: int(FEET_TO_METERS * k))
data.head()

,state,peak,elev_ft,elev_m
0,Alabama,Cheaha Mountain,2405,733
1,Alaska,Denali,20320,6193
2,Arizona,Humphreys Peak,12633,3850
3,Arkansas,Magazine Mountain,2753,839
4,California,Mount Whitney,14495,4418


Note the use of the anonymous function (denoted by the keyword `lambda`) passed to the `apply` method.

We can also **create a column as a function of other columns**:

In [82]:
data['scale_factor'] = data.elev_m / data.elev_ft
data.head()

,state,peak,elev_ft,elev_m,scale_factor
0,Alabama,Cheaha Mountain,2405,733,0.304782
1,Alaska,Denali,20320,6193,0.304774
2,Arizona,Humphreys Peak,12633,3850,0.304757
3,Arkansas,Magazine Mountain,2753,839,0.304758
4,California,Mount Whitney,14495,4418,0.304795


## 4. manipulating data

Now we're ready for a little more pandas power. Just about anything you can do in a relational database can be done in pandas too (and with less overhead).

Let's look at the dataset in `fish.csv`. This dataset contains a number of measurements for a sample of 159 fish from 7 different species.

In [83]:
FISH_FILE = '~/gits/gads_26/datasets/fish.csv'
fish = pd.read_csv(FISH_FILE)
fish.head()

,Obs,Weight,Length1,Length2,Length3,Height%,Width%,Sex,Species
0,1,242,23.2,25.4,30.0,38.4,13.4,NaN,Abramis brama
1,2,290,24.0,26.3,31.2,40.0,13.8,NaN,Abramis brama
2,3,340,23.9,26.5,31.1,39.8,15.1,NaN,Abramis brama
3,4,363,26.3,29.0,33.5,38.0,13.3,NaN,Abramis brama
4,5,430,26.5,29.0,34.0,36.6,15.1,NaN,Abramis brama


Suppose we're interested in finding the mean weight by species. This requires us to perform a **bucketed calculation**, where the buckets are species and the calculation is finding the mean (a better way to say this is that we're calculating **conditional means**, where means are calculated conditional on a given species).

If we were using a relational database, you could imagine using SQL's `GROUP BY` syntax to get at the answer. We can do the same thing with pandas:

In [84]:
fish.groupby('Species').Weight.mean()

Species
Abramis bjrkna       154.818182
Abramis brama        626.000000
Esox lucius          718.705882
Leuciscus rutilus    152.050000
Leusiscus idus       531.000000
Osmerus eperlanus     11.178571
Perca fluviatilis    382.239286
Name: Weight, dtype: float64

This is just about as easy as it looks. The only subtlety has to do with the types that pandas sends the data through before it presents the result:

In [85]:
fish.groupby('Species')

As you can see, the `groupby` method returns an object of type `DataFrameGroupBy`. This doesn't give us much until we apply an [aggregation function](http://pandas.pydata.org/pandas-docs/stable/groupby.html), in this case `mean`. It's good to know this to avoid getting stumped by it later, but we don't have to worry much about these details.

Another important piece of relational algebra is performed by SQL's [`JOIN`](http://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins) syntax. We can do these in pandas too.

First let's load a couple of datasets to join. The `cp_temps_mini.tsv` file contains average annual temperatures recorded in Central Park for the years 1918-40, and `snowfall.tsv` contains total annual snowfall for two cities for the years 1918-37.

In [90]:
TEMPS_FILE = '~/gits/gads_26/datasets/cp_temps_mini.tsv'
SNOW_FILE = '~/gits/gads_26/datasets/snowfall.tsv'

temps = pd.read_csv(TEMPS_FILE, sep='\t')
snow = pd.read_csv(SNOW_FILE, sep='\t')

temps.head()

,date,cp_avg_temp
0,1918,53.2
1,1919,53.8
2,1920,52.3
3,1921,54.9
4,1922,53.5


In [91]:
snow.head()

,date,"Buffalo, NY","Cairo, IL"
0,1918,25.0,1.8
1,1919,69.4,4.5
2,1920,53.5,13.9
3,1921,39.8,4.0
4,1922,63.6,1.2


DataFrames do have a method called `join`, but the `merge` method gives us a bit of syntactic sugar to make things smoother:

In [92]:
temps.merge(snow).head()

,date,cp_avg_temp,"Buffalo, NY","Cairo, IL"
0,1918,53.2,25.0,1.8
1,1919,53.8,69.4,4.5
2,1920,52.3,53.5,13.9
3,1921,54.9,39.8,4.0
4,1922,53.5,63.6,1.2


In [ ]:
Look at what happens at the tail end of the dataset:

In [93]:
temps.merge(snow).tail()

,date,cp_avg_temp,"Buffalo, NY","Cairo, IL"
15,1933,54.2,71.8,2.7
16,1934,53.0,49.1,1.6
17,1935,53.1,103.9,14.1
18,1936,53.4,51.6,5.4
19,1937,54.5,81.6,3.0


Even though the temps dataset contains records all the way out to 1940, our joined dataset is truncated at 1937 because that's where the snowfall records end. This is an example of an **inner join**, which is the default behavior of the `merge` method.

We can change the behavior by being explicit:

In [94]:
temps.merge(snow, how='outer').tail()

,date,cp_avg_temp,"Buffalo, NY","Cairo, IL"
18,1936,53.4,51.6,5.4
19,1937,54.5,81.6,3.0
20,1938,55.2,NaN,NaN
21,1939,54.7,NaN,NaN
22,1940,51.9,NaN,NaN


Using an **outer join** produces nearly the same results as before, but this time the extra rows from `snowfall.tsv` that don't have matching join keys in `cp_temps_mini.tsv` are included, and blank values (`NaN`) are appended to the shorter dataset.